# ELMo

ELMo（Embeddings from Language Models）是一种由Allen Institute for AI的研究者Matthew E. Peters等在2018年提出的embedding方法，通过上下文中的词语来捕捉语义。ELMo的作者认为之前的文本向量化方式没有考虑一个词在不同上下文中意义的变化，所以与Word2Vec和GloVe不同，ELMo能够捕捉到词在不同上下文中的不同意义，从而生成更为丰富和精确的embedding。

ELMo的提出意味着我们从词嵌入（Word Embedding）时代进入了语境词嵌入（Contextualized Word-Embedding）时代！（**解决了一词多义的问题**）

> 对以往的word2vec及GloVe等而言，ELMo迈出了具有历史意义的一步。Word Embedding本质是一种静态的嵌入方法，所谓静态指的是模型训练好之后每个单词的表达就固定了，在后续使用时，无论新句子的上下文单词是什么，这个单词的Word Embedding都不会随着上下文场景的变化而变化。但这种局限性在很多场景是致命的，因为一个单词的语义往往会因其语境不同而不同。以“苹果”为例，在水果语境中，它是指可吃的苹果，但在电子产品语境中，它是指苹果公司的手机iPhone。
>

ELMo使用双向LSTM（Long Short-Term Memory）网络作为基础架构。该网络包括一个向前的LSTM和一个向后的LSTM，它们分别从左到右和从右到左遍历文本序列，学习每个词的前向和后向上下文。这个双向LSTM被训练为一个语言模型，目标是预测句子中的下一个词（对于前向LSTM）和前一个词（对于后向LSTM）。对于给定的文本序列，ELMo会为每个词提取多层的表示。对于每个词，ELMo提供三层的embedding输出：一层来自embedding层（类似于传统的embedding），另外两层来自双向LSTM的各自输出。最后，ELMo使用这些层的加权和作为结果，公式如下：
$$
{ELMO}_t = \gamma \sum_{j=0}^L s_j h_{t}^{(j)}
$$
其中 $\gamma$ 是一个可学习的标量；$s_j$是层$j$的可学习权重；$h_t^{(j)}$是词 $t$ 在层 $j$ 的表示。为了使这些输出被应用在不同任务中，ELMo提出了一种动态加权的方法，可以为不同层的输出分配不同的权重。这样，模型不仅使用了所有层的信息，还根据任务动态调整了每层信息的重要性。


## ELMo的整体模型结构

<img src="images/img_14.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

## ELMo模型的处理流程

EMLo的实现主要涉及语言模型（Language Model）

它首先把输入转换为字符级别的Embedding，根据字符级别的Embedding来生成上下文无关的Word Embedding，然后使用双向语言模型（如Bi-LSTM）生成上下文相关的Word Embedding。

### 1.输入句子
句子维度为 $B*W*C$ ，其中 $B$ 表示批量大小（batch_size），$W$ 表示一句话中的单词数num_words，$C$ 表示每个单词的最大字符数目（max_characters_per_token），可设置为某个固定值（如50或60）。在一个批量中，语句有长短，可以采用Padding方法对齐

### 2.字符编码层（字符级embedding）

输入语句首先经过一个字符编码层（Char Encode Layer），因为ELMo实际上是基于字符（char）的，所以它会先对每个单词中的所有字符进行编码，从而得到这个单词的表示。因此经过字符编码层编码后的数据的维度为$B*W*D$，这就是我们熟知的对于一个句子在字符级别上的维度。输入度量是字符而不是词汇，以便模型能捕捉词的内部结构信息。比如beauty和beautiful，即使不了解这两个词的上下文，双向语言模型也能够识别出它们在一定程度上的相关性。

<img src="images/img_15.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

### 3.双向语言模型（Bi-LSTM）

对字符级语句编码后，该句子会经过双向语言模型（Bi-LSTM），模型内部先分开训练了两个正向和反向的语言模型，而后将其表征进行拼接，最终得到的输出维度为 $（L+1）*B*W*2D$。这里 +1 实际上是加上了最初的Embedding层，有点儿像残差连接。

<img src="images/img_16.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

### 4.混合层（Mix Layer）

得到各个层的表征之后，会经过一个混合层（Scalar Mixer），它会对前面这些层的表示进行线性融合，得出最终的ELMo向量，维度为 $(B，W，2D)$

## 优缺点

优点：

● 基于整个句子的上下文信息生成embedding，能够更好地处理多义词和上下文依赖性强的词。\
● 能够捕捉复杂的句子结构和长距离依赖关系。\
● 预训练的ELMo模型可以很容易地迁移到各种下游自然语言处理任务中。\
● 实现从单纯的词嵌入（Word Embedding）到情景词嵌入（Contextualized Word Embedding）的转变\
● 实现预训练模型从静态到动态的转变


缺点：

● 模型较大，训练和推理过程需要大量的计算资源。\
● 由于模型复杂，推理速度较慢。\
● ELMo预训练模型的特征提取器使用了双向循环神经网络（如Bi-LSTM），循环神经网络的训练需要按序列从左到右或从右到左，严格限制了并发处理能力
> 由于ELMo采用LSTM架构，因此，模型的并发能力、关注语句的长度等在大的语料库面前，有点力不从心
>
● ELMo的每一层会拼接两个方向的向量，所以这种操作实际仍然属于单向学习，无法做到同时向两个方向学习



## 应用场景

● 情感分析
● 问答系统
● 文本分类
● 机器翻译


通过上面的文档介绍，我们已经了解到了ELMo的原理，并且也了解到了如何使用ELMo进行情感分析。然而，ELMo并不是一个简单的模型，它需要大量的训练数据才能达到很好的效果。因此，在应用ELMo时，需要根据具体的任务和数据情况来确定是否需要使用ELMo。

**仔细查看其中的代码，相信我，你收获会很大！！👨‍💻**

以下是关于ELMO的训练代码：[ELMO CODE](https://github.com/dbiir/UER-py/wiki/%E9%A2%84%E8%AE%AD%E7%BB%83%E6%A8%A1%E5%9E%8B%E4%BD%BF%E7%94%A8%E7%A4%BA%E4%BE%8B#elmo)


```shell
python3 preprocess.py --corpus_path corpora/book_review.txt --vocab_path models/google_zh_vocab.txt \
                      --dataset_path dataset.pt --processes_num 8 --data_processor bilm

python3 pretrain.py --dataset_path dataset.pt --vocab_path models/google_zh_vocab.txt  \
                    --config_path models/rnn/bilstm_config.json \
                    --output_model_path models/output_model.bin \
                    --world_size 8 --gpu_ranks 0 1 2 3 4 5 6 7 --learning_rate 5e-4 \
                    --data_processor bilm \
                    --embedding word --remove_embedding_layernorm --encoder bilstm --target bilm
```

以上代码训练了一个ELMo模型，并且使用了一个名为bilm的数据处理器。bilm数据处理器会将输入的文本数据转换为字符级别的表示，然后使用双向语言模型（Bi-LSTM）生成上下文相关的词表示。

保存的模型是模型的**权重参数**，可以通过torch.load()函数加载。

预训练模型能够产生高质量的词向量。传统的词向量（比如word2vec和GloVe）给定一个单词固定的向量（上下文无关向量）。然而，一词多义是人类语言中的常见现象。一个单词的意思依赖于其上下文。我们可以使用预训练模型的隐层去表示单词。值得注意的是大多数的中文预训练模型是基于字的。如果需要真正的词向量而不是字向量，

上下文相关词向量以词搜词（文本相似度）：[文本相似度计算](https://github.com/dbiir/UER-py/blob/master/scripts/topn_words_dep.py)


```shell
python3 scripts/topn_words_dep.py --load_model_path models/google_zh_model.bin --vocab_path models/google_zh_vocab.txt \
                                  --cand_vocab_path models/google_zh_vocab.txt --test_path target_words_with_sentences.txt --config_path models/bert/base_config.json \
                                  --batch_size 256 --seq_length 32 --tokenizer bert

python3 scripts/topn_words_dep.py --load_model_path models/bert_wiki_word_model.bin --vocab_path models/wiki_word_vocab.txt \
                                  --cand_vocab_path models/wiki_word_vocab.txt --test_path target_words_with_sentences.txt --config_path models/bert/base_config.json \
                                  --batch_size 256 --seq_length 32 --tokenizer space